In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from pandas.api.types import is_numeric_dtype
from functools import reduce
from pandas.api.types import is_numeric_dtype
from itertools import groupby

In [2]:
df = pd.read_csv('../data/Iris/Iris-35-attcon.csv')

In [3]:
appr_df = df.copy()

In [4]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,-,-,Iris-setosa
1,-,3,-,-,Iris-setosa
2,4.7,3.2,1.3,-,Iris-setosa
3,4.6,-,1.5,-,Iris-setosa
4,-,-,1.4,0.2,Iris-setosa
5,-,-,-,0.4,Iris-setosa
6,4.6,3.4,1.4,-,Iris-setosa
7,-,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,-,Iris-setosa
9,4.9,3.1,-,-,Iris-setosa


### Handling Missing attributes using attribute-concept values !!

#### Defining Goal Set !!

In [5]:
df_headers = list(df)
concept = df_headers[-1]
concept

'class'

In [6]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [7]:
#calculating cases by concepts and making sets
U = [] #universal list containing all cases
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        U.append(index+1)
        if row[concept] == item:
            temp_list.append(index)
    goal_list.append(temp_list)
    temp_list = []

In [8]:
print(goal_list,)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]]


#### Building Case List !!

In [9]:
attributes = list(df)
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [10]:
case_list = []

In [11]:
def discretize(numeric_col):
    print(numeric_col)
    #Sorting the values of numeric column
    sort_col = df[numeric_col].sort_values()
   
    df['sort_col'] = sort_col.values
    point_list = df['sort_col'].unique()
    point_list = list(point_list)
    if '-' in point_list:
        point_list.remove('-')
    print(point_list)
    
    #Finding average between each two points
    avg_list = []
    for i in range(len(point_list)-1):
        avg = (float(point_list[i]) + float(point_list[i+1]))/2
        avg_list.append(round(float(avg),2))
        
    print (avg_list)
    #Performing the discretization and adding the cases
    for i in avg_list:
        case = str(numeric_col) + "," + str(round(float(point_list[0]),1)) + ".." + str(i)
        case2 = str(numeric_col) + "," + str(i) + ".." + str(round(float(point_list[len(point_list)-1]),1))
        case_list.append(case)
        case_list.append(case2)

In [12]:
#Discretization considering upto 2 decimal point
for item in attributes[:-1]:
        discretize(item)

sepal_length
['4.3', '4.4', '4.6', '4.7', '4.8', '4.9', '5', '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.7', '5.8', '5.9', '6', '6.1', '6.2', '6.3', '6.4', '6.5', '6.6', '6.7', '6.8', '6.9', '7.1', '7.2', '7.3', '7.7', '7.9']
[4.35, 4.5, 4.65, 4.75, 4.85, 4.95, 5.05, 5.15, 5.25, 5.35, 5.45, 5.55, 5.65, 5.75, 5.85, 5.95, 6.05, 6.15, 6.25, 6.35, 6.45, 6.55, 6.65, 6.75, 6.85, 7.0, 7.15, 7.25, 7.5, 7.8]
sepal_width
['2.2', '2.3', '2.4', '2.5', '2.6', '2.7', '2.8', '2.9', '3', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '4.1']
[2.25, 2.35, 2.45, 2.55, 2.65, 2.75, 2.85, 2.95, 3.05, 3.15, 3.25, 3.35, 3.45, 3.55, 3.65, 3.75, 3.95]
petal_length
['1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.9', '3', '3.3', '3.5', '3.6', '3.7', '3.8', '3.9', '4', '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.8', '4.9', '5', '5.1', '5.2', '5.3', '5.5', '5.6', '5.7', '5.8', '5.9', '6', '6.1', '6.3', '6.4', '6.6', '6.7']
[1.1, 1.25, 1.35, 1.45, 1.55, 1.65, 1.8, 2.45, 3.15, 3.4, 3.55, 3.65, 3.

In [13]:
case_list

['sepal_length,4.3..4.35',
 'sepal_length,4.35..7.9',
 'sepal_length,4.3..4.5',
 'sepal_length,4.5..7.9',
 'sepal_length,4.3..4.65',
 'sepal_length,4.65..7.9',
 'sepal_length,4.3..4.75',
 'sepal_length,4.75..7.9',
 'sepal_length,4.3..4.85',
 'sepal_length,4.85..7.9',
 'sepal_length,4.3..4.95',
 'sepal_length,4.95..7.9',
 'sepal_length,4.3..5.05',
 'sepal_length,5.05..7.9',
 'sepal_length,4.3..5.15',
 'sepal_length,5.15..7.9',
 'sepal_length,4.3..5.25',
 'sepal_length,5.25..7.9',
 'sepal_length,4.3..5.35',
 'sepal_length,5.35..7.9',
 'sepal_length,4.3..5.45',
 'sepal_length,5.45..7.9',
 'sepal_length,4.3..5.55',
 'sepal_length,5.55..7.9',
 'sepal_length,4.3..5.65',
 'sepal_length,5.65..7.9',
 'sepal_length,4.3..5.75',
 'sepal_length,5.75..7.9',
 'sepal_length,4.3..5.85',
 'sepal_length,5.85..7.9',
 'sepal_length,4.3..5.95',
 'sepal_length,5.95..7.9',
 'sepal_length,4.3..6.05',
 'sepal_length,6.05..7.9',
 'sepal_length,4.3..6.15',
 'sepal_length,6.15..7.9',
 'sepal_length,4.3..6.25',
 's

#### Fill up missing values !!

In [14]:
df2 = df.copy()

In [15]:
for col in df:
    for i, row_value in df[col].iteritems():
        
        #This is for attribute-concept value interpretation
        #For Lost value interpretation, we don't add that case to any block at all
        if row_value == '-':
            
            final_values = []
            print(col,i)
            concept_tmp = df[concept][i]
            print(concept_tmp)
            
            for item_list in goal_list:
                if i in item_list:
                    final_values = item_list
                    
            print (final_values)
            
            #Taking out all unique cases corresponding to the matched concept
            col_values = []
            for item in final_values:
                col_values.append(df[col][item])
        
            
            uniqueList = []
            for letter in col_values:
                if letter not in uniqueList:
                    uniqueList.append(letter)
            
            if '-' in uniqueList:
                uniqueList.remove('-')
                
            if '?' in uniqueList:
                uniqueList.remove('?')
            print(uniqueList)
            
#             print(df.loc[i,col])
            df2.at[i, col] = uniqueList
            
    print("\n")

sepal_length 1
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '4.4', '4.9', '5.4', '4.8', '4.3', '5.8', '5', '5.2', '5.5', '5.3']
sepal_length 4
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '4.4', '4.9', '5.4', '4.8', '4.3', '5.8', '5', '5.2', '5.5', '5.3']
sepal_length 5
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['5.1', '4.7', '4.6', '4.4', '4.9', '5.4', '4.8', '4.3', '5.8', '5', '5.2', '5.5', '5.3']
sepal_length 7
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14

['6.9', '5.5', '6.5', '5.7', '4.9', '6.6', '5.2', '5.9', '6.7', '5.6', '5.8', '6.2', '6.1', '6.8', '6', '5.4', '5', '5.1']
sepal_length 92
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['6.9', '5.5', '6.5', '5.7', '4.9', '6.6', '5.2', '5.9', '6.7', '5.6', '5.8', '6.2', '6.1', '6.8', '6', '5.4', '5', '5.1']
sepal_length 94
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['6.9', '5.5', '6.5', '5.7', '4.9', '6.6', '5.2', '5.9', '6.7', '5.6', '5.8', '6.2', '6.1', '6.8', '6', '5.4', '5', '5.1']
sepal_length 95
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 

['3.5', '3', '3.2', '3.4', '2.9', '3.1', '3.7', '3.8', '3.6', '3.3', '4.1', '2.3']
sepal_width 16
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['3.5', '3', '3.2', '3.4', '2.9', '3.1', '3.7', '3.8', '3.6', '3.3', '4.1', '2.3']
sepal_width 20
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['3.5', '3', '3.2', '3.4', '2.9', '3.1', '3.7', '3.8', '3.6', '3.3', '4.1', '2.3']
sepal_width 27
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['3.5', '3', '3.2', '3.4', '2.9', '3.1', '3.7', '3.8', '3.6', '3.3', '4.1', '2.3']
sepal_width 31

['3.2', '3.1', '2.8', '2.4', '2.7', '3', '2.2', '2.9', '2.6', '2.3']
sepal_width 85
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['3.2', '3.1', '2.8', '2.4', '2.7', '3', '2.2', '2.9', '2.6', '2.3']
sepal_width 87
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['3.2', '3.1', '2.8', '2.4', '2.7', '3', '2.2', '2.9', '2.6', '2.3']
sepal_width 89
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['3.2', '3.1', '2.8', '2.4', '2.7', '3', '2.2', '2.9', '2.6', '2.3']
sepal_width 95
Iris-versicol

petal_length 93
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['4.7', '4.9', '4', '4.6', '4.5', '3.3', '3.9', '3.5', '4.2', '3.6', '4.1', '4.8', '4.3', '4.4', '5', '3.8', '3.7', '3']
petal_length 96
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['4.7', '4.9', '4', '4.6', '4.5', '3.3', '3.9', '3.5', '4.2', '3.6', '4.1', '4.8', '4.3', '4.4', '5', '3.8', '3.7', '3']
petal_length 97
Iris-versicolor
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
['4.7', '4.9', '4', '4.6', '4.5', '3.3', '3.9', '3.5', '4.2

['0.2', '0.4', '0.1', '0.3', '0.5', '0.6']
petal_width 31
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['0.2', '0.4', '0.1', '0.3', '0.5', '0.6']
petal_width 32
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['0.2', '0.4', '0.1', '0.3', '0.5', '0.6']
petal_width 33
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
['0.2', '0.4', '0.1', '0.3', '0.5', '0.6']
petal_width 35
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 

Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[]
sort_col 38
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[]
sort_col 39
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[]
sort_col 40
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[]
sort_col 41
Iris-setosa
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3

In [16]:
df2

,sepal_length,sepal_width,petal_length,petal_width,class,sort_col
0,5.1,3.5,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
1,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
2,4.7,3.2,1.3,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
3,4.6,"[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.5,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
4,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.4,0.2,Iris-setosa,[]
5,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....","[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]",0.4,Iris-setosa,[]
6,4.6,3.4,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
7,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3.4,1.5,0.2,Iris-setosa,[]
8,4.4,2.9,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
9,4.9,3.1,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]


In [17]:
df = df2.copy()

In [18]:
df_headers = list(df)
concept = df_headers[-2]
concept_list = df[concept].unique()

In [19]:
concept_list

array(['Iris-setosa', 'Iris-versicolor', 'Iris-viginica'], dtype=object)

In [20]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [21]:
print(goal_list,)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], [51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]]


In [22]:
df

,sepal_length,sepal_width,petal_length,petal_width,class,sort_col
0,5.1,3.5,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
1,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
2,4.7,3.2,1.3,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
3,4.6,"[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.5,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
4,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.4,0.2,Iris-setosa,[]
5,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....","[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]",0.4,Iris-setosa,[]
6,4.6,3.4,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
7,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3.4,1.5,0.2,Iris-setosa,[]
8,4.4,2.9,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]
9,4.9,3.1,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa,[]


#### Building blocks for attribute-value pair !!

In [23]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) >= float(start) and float(case) <= float(end):
                        temp_list.append(index+1)
                   
                
            else:
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
                
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == b:
                        temp_list.append(index+1)
            else:           
                if float(row[a]) == b:
                    temp_list.append(index+1)
        
        temp_list = list(set(temp_list))
        att_val_list.append(temp_list)
        temp_list = []

In [24]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       sepal_length,4.3..4.35[2, 34, 35, 5, 6, 36, 8, 39, 41, 42, 12, 43, 14, 45, 16, 47, 19, 23, 27, 29]
1       sepal_length,4.35..7.9[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]
2       sepal_length,4.3..4.5[2, 5, 6, 8, 9, 12, 14, 16, 19, 23, 27, 29, 34, 35, 36, 39, 41, 42, 43, 45, 47]
3       sepal_length,4.5..7.9[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 2

In [25]:
#Creating data for case and att-value list
data = {'Cases': case_list, 'att_val': att_val_list}

In [26]:
df2 = pd.DataFrame(data)

In [27]:
#Cases and corresponding att-value pairs
df2

,Cases,att_val
0,"sepal_length,4.3..4.35","[2, 34, 35, 5, 6, 36, 8, 39, 41, 42, 12, 43, 1..."
1,"sepal_length,4.35..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."
2,"sepal_length,4.3..4.5","[2, 5, 6, 8, 9, 12, 14, 16, 19, 23, 27, 29, 34..."
3,"sepal_length,4.5..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1..."
4,"sepal_length,4.3..4.65","[2, 4, 5, 6, 7, 8, 9, 12, 14, 16, 19, 23, 27, ..."
5,"sepal_length,4.65..7.9","[1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17,..."
6,"sepal_length,4.3..4.75","[2, 3, 4, 5, 6, 7, 8, 9, 12, 14, 16, 19, 23, 2..."
7,"sepal_length,4.75..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18..."
8,"sepal_length,4.3..4.85","[2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 16, 19, 2..."
9,"sepal_length,4.85..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 15, 16, 17, 18, 19..."


### Start of Lower/Upper & Probabilistic Approximations !!

#### Building Chacteristic Set !!

In [28]:
attributes = list(appr_df)
del attributes[-1]

In [29]:
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [30]:
appr_df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,-,-,Iris-setosa
1,-,3,-,-,Iris-setosa
2,4.7,3.2,1.3,-,Iris-setosa
3,4.6,-,1.5,-,Iris-setosa
4,-,-,1.4,0.2,Iris-setosa
5,-,-,-,0.4,Iris-setosa
6,4.6,3.4,1.4,-,Iris-setosa
7,-,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,-,Iris-setosa
9,4.9,3.1,-,-,Iris-setosa


In [31]:
case_list = []
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes:
    print(item)
    
    #check for non numeric columns
    if not is_numeric_dtype(appr_df[item]):
        temp = appr_df[item].unique()
        print(temp)
        for i in temp:
            if i == '?' or i == '-':
                continue
            else:
                case = item + "," + i
                case_list.append(case)

sepal_length
['5.1' '-' '4.7' '4.6' '4.4' '4.9' '5.4' '4.8' '4.3' '5.8' '5' '5.2' '5.5'
 '5.3' '6.9' '6.5' '5.7' '6.6' '5.9' '6.7' '5.6' '6.2' '6.1' '6.8' '6'
 '6.3' '7.1' '7.3' '7.2' '7.7' '6.4' '7.9']
sepal_width
['3.5' '3' '3.2' '-' '3.4' '2.9' '3.1' '3.7' '3.8' '3.6' '3.3' '4.1' '2.3'
 '2.8' '2.4' '2.7' '2.2' '2.6' '2.5']
petal_length
['-' '1.3' '1.5' '1.4' '1.2' '1.7' '1' '1.9' '1.6' '4.7' '4.9' '4' '4.6'
 '4.5' '3.3' '3.9' '3.5' '4.2' '3.6' '4.1' '4.8' '4.3' '4.4' '5' '3.8'
 '3.7' '3' '6' '5.1' '5.9' '5.6' '6.6' '6.3' '5.8' '5.3' '6.7' '5.7' '6.1'
 '6.4' '5.5' '5.2']
petal_width
['-' '0.2' '0.4' '0.1' '0.3' '0.5' '0.6' '1.5' '1.3' '1' '1.4' '1.1' '1.8'
 '1.2' '1.6' '2.5' '1.9' '2.1' '2.2' '1.7' '2' '2.3' '2.4']


In [32]:
case_list

['sepal_length,5.1',
 'sepal_length,4.7',
 'sepal_length,4.6',
 'sepal_length,4.4',
 'sepal_length,4.9',
 'sepal_length,5.4',
 'sepal_length,4.8',
 'sepal_length,4.3',
 'sepal_length,5.8',
 'sepal_length,5',
 'sepal_length,5.2',
 'sepal_length,5.5',
 'sepal_length,5.3',
 'sepal_length,6.9',
 'sepal_length,6.5',
 'sepal_length,5.7',
 'sepal_length,6.6',
 'sepal_length,5.9',
 'sepal_length,6.7',
 'sepal_length,5.6',
 'sepal_length,6.2',
 'sepal_length,6.1',
 'sepal_length,6.8',
 'sepal_length,6',
 'sepal_length,6.3',
 'sepal_length,7.1',
 'sepal_length,7.3',
 'sepal_length,7.2',
 'sepal_length,7.7',
 'sepal_length,6.4',
 'sepal_length,7.9',
 'sepal_width,3.5',
 'sepal_width,3',
 'sepal_width,3.2',
 'sepal_width,3.4',
 'sepal_width,2.9',
 'sepal_width,3.1',
 'sepal_width,3.7',
 'sepal_width,3.8',
 'sepal_width,3.6',
 'sepal_width,3.3',
 'sepal_width,4.1',
 'sepal_width,2.3',
 'sepal_width,2.8',
 'sepal_width,2.4',
 'sepal_width,2.7',
 'sepal_width,2.2',
 'sepal_width,2.6',
 'sepal_width,2

In [33]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in appr_df.iterrows():
            if row[a] >= float(start) and row[a] <= float(end):
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in appr_df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if case == b:
                        temp_list.append(index+1)
                        
            if row[a] == b:
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []

[1, 18, 20, 22, 24, 40, 99]
[3, 30]
[4, 7, 48]
[9]
[10, 38, 58, 107]
[11, 17, 21, 32, 85]
[13, 25, 31, 46]
[14]
[15, 68, 83, 102]
[26, 44, 50, 94]
[28, 33, 60]
[37, 54, 82]
[49]
[53, 121, 140]
[55, 105, 117, 148]
[56, 80, 97, 100]
[59]
[62, 71, 150]
[66, 78, 87, 141, 146]
[67, 89, 122]
[69, 98, 127, 149]
[72, 92, 128, 135]
[77, 113, 144]
[84, 86, 120]
[101, 104, 134, 137, 147]
[103]
[108]
[110, 126, 130]
[118, 119, 123, 136]
[129]
[132]
[1, 18, 41, 44]
[2, 13, 14, 26, 39, 46, 62, 78, 89, 92, 106, 113, 128, 130, 136, 139, 146, 148]
[3, 30, 36, 48, 51, 71, 111, 126, 144]
[7, 8, 12, 25, 27, 29, 137, 149]
[9, 64, 65, 75, 97, 98, 104]
[10, 31, 35, 53, 66, 87, 138, 140, 141]
[11, 22]
[19, 20, 118, 132]
[23, 110]
[24, 101]
[33]
[42, 94]
[55, 56, 72, 74, 100, 122, 127, 129, 131, 133]
[58]
[60, 84, 95, 102, 143]
[63, 69, 120]
[80, 91, 93, 119, 135]
[107, 109, 114]
[3, 17, 41, 43]
[4, 8, 11, 20, 28, 32, 33, 38, 40, 49]
[5, 7, 9, 13, 18, 29, 34, 46, 48, 50]
[15, 36]
[19, 24]
[23]
[25]
[26, 27, 44

In [34]:
df = df.drop(columns=['sort_col'])

In [35]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
1,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
2,4.7,3.2,1.3,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
3,4.6,"[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.5,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
4,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....",1.4,0.2,Iris-setosa
5,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...","[3.5, 3, 3.2, 3.4, 2.9, 3.1, 3.7, 3.8, 3.6, 3....","[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]",0.4,Iris-setosa
6,4.6,3.4,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
7,"[5.1, 4.7, 4.6, 4.4, 4.9, 5.4, 4.8, 4.3, 5.8, ...",3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,"[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa
9,4.9,3.1,"[1.3, 1.5, 1.4, 1.2, 1.7, 1, 1.9, 1.6]","[0.2, 0.4, 0.1, 0.3, 0.5, 0.6]",Iris-setosa


In [36]:
U = set(U)
print(U,)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150}


In [37]:
# Creating dictionary combining case_list and att_val list
block = dict(zip(case_list, att_val_list))
block

{'sepal_length,5.1': [1, 18, 20, 22, 24, 40, 99],
 'sepal_length,4.7': [3, 30],
 'sepal_length,4.6': [4, 7, 48],
 'sepal_length,4.4': [9],
 'sepal_length,4.9': [10, 38, 58, 107],
 'sepal_length,5.4': [11, 17, 21, 32, 85],
 'sepal_length,4.8': [13, 25, 31, 46],
 'sepal_length,4.3': [14],
 'sepal_length,5.8': [15, 68, 83, 102],
 'sepal_length,5': [26, 44, 50, 94],
 'sepal_length,5.2': [28, 33, 60],
 'sepal_length,5.5': [37, 54, 82],
 'sepal_length,5.3': [49],
 'sepal_length,6.9': [53, 121, 140],
 'sepal_length,6.5': [55, 105, 117, 148],
 'sepal_length,5.7': [56, 80, 97, 100],
 'sepal_length,6.6': [59],
 'sepal_length,5.9': [62, 71, 150],
 'sepal_length,6.7': [66, 78, 87, 141, 146],
 'sepal_length,5.6': [67, 89, 122],
 'sepal_length,6.2': [69, 98, 127, 149],
 'sepal_length,6.1': [72, 92, 128, 135],
 'sepal_length,6.8': [77, 113, 144],
 'sepal_length,6': [84, 86, 120],
 'sepal_length,6.3': [101, 104, 134, 137, 147],
 'sepal_length,7.1': [103],
 'sepal_length,7.3': [108],
 'sepal_length,7.2

In [38]:
attributes = list(df)
attributes

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [39]:
dic = {}
for index, row in df.iterrows():
    tmp_set = set()
    final_union = []
    char_list = []
    char_list_2 = []
    final_union_set = []
  
    for cols in attributes[:-1]:
        #If the value for corresponding attribute is a list then create all of the att-value pairs
        if type(df.loc[index,cols]) == list:
            print("When values are list") #########
            for item in df.loc[index,cols]:
                block_key = cols + "," + item
                char_list.append(block_key) #char_list has all att-val cases
                print(char_list) #########
                
            union_set = set()
            #Compute union of att-concept value case
            for item in char_list:
                union_set = union_set.union(set(block[item]))
            
            print("Union Set: ", union_set) #########
            final_union.append(union_set)
        
        else:
            print("When value is single") #########
            block_key = cols + "," + df.loc[index,cols]
            char_list_2.append(block_key) #char_list_2 has all single cases
            print(char_list_2) ##########
   
    #Compute instersection for this current row for Characteristics set
    
    print("final_union: ", final_union) ########
    if len(final_union):
        final_union_set = list(reduce(set.intersection, [set(item) for item in final_union]))
        
    
    print("Final Union Set: ", final_union_set) ##########
    
    for item in char_list_2:
        if item in block:
            print("When item is found as key in the block") ########
            if tmp_set == set():
                #Copy over the current set elements to B
                for i in range(len(block[item])):
                    tmp_set.add(block[item][i])
                    
            tmp_set = tmp_set.intersection(set(block[item]))
            print(tmp_set) ########
        
        #If item not in block
        else:
            print("When item is not found as key in the block") ########
            print(tmp_set) ########
            print(U) #########
            if tmp_set == set():
                tmp_set = U
                
            tmp_set = tmp_set.intersection(U)
            print(tmp_set) ########
    
    print("Final tmp_set: ", tmp_set) #########
    final_union_set = set(final_union_set)
    
    if final_union_set == set():
        tmp_set = tmp_set
    else:
        tmp_set = tmp_set.intersection(final_union_set)
        
    print("This is the final value: ", tmp_set) ###########
        
    key = ('K_%d' % (index+1))
    print(key) #########
    dic[key] = tmp_set
    print("\n") ##########

When value is single
['sepal_length,5.1']
When value is single
['sepal_length,5.1', 'sepal_width,3.5']
When values are list
['petal_length,1.3']
['petal_length,1.3', 'petal_length,1.5']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7', 'petal_length,1']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7', 'petal_length,1', 'petal_length,1.9']
['petal_length,1.3', 'petal_length,1.5', 'petal_length,1.4', 'petal_length,1.2', 'petal_length,1.7', 'petal_length,1', 'petal_length,1.9', 'petal_length,1.6']
Union Set:  {3, 4, 5, 7, 8, 9, 11, 13, 15, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 36, 38, 40, 41, 43, 44, 46, 47, 48, 49, 50}
When values

['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,4.4']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,4.4', 'sepal_length,4.9']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,4.4', 'sepal_length,4.9', 'sepal_length,5.4']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,4.4', 'sepal_length,4.9', 'sepal_length,5.4', 'sepal_length,4.8']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,4.4', 'sepal_length,4.9', 'sepal_length,5.4', 'sepal_length,4.8', 'sepal_length,4.3']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,4.4', 'sepal_length,4.9', 'sepal_length,5.4', 'sepal_length,4.8', 'sepal_length,4.3', 'sepal_length,5.8']
['sepal_length,5.1', 'sepal_length,4.7', 'sepal_length,4.6', 'sepal_length,4.4', 'sepal_length,4.9', 'sepal_length,5.4', 'sepal_length,4.8', 'sepal_lengt

Union Set:  {1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 18, 19, 20, 22, 23, 24, 25, 26, 27, 29, 30, 31, 33, 35, 36, 39, 41, 42, 44, 46, 48, 51, 53, 62, 64, 65, 66, 71, 75, 78, 87, 89, 92, 94, 97, 98, 101, 104, 106, 110, 111, 113, 118, 126, 128, 130, 132, 136, 137, 138, 139, 140, 141, 144, 146, 148, 149}
When value is single
['sepal_length,5.4', 'petal_length,1.5']
When values are list
['sepal_width,3.5', 'sepal_width,3', 'sepal_width,3.2', 'sepal_width,3.4', 'sepal_width,2.9', 'sepal_width,3.1', 'sepal_width,3.7', 'sepal_width,3.8', 'sepal_width,3.6', 'sepal_width,3.3', 'sepal_width,4.1', 'sepal_width,2.3', 'petal_width,0.2']
['sepal_width,3.5', 'sepal_width,3', 'sepal_width,3.2', 'sepal_width,3.4', 'sepal_width,2.9', 'sepal_width,3.1', 'sepal_width,3.7', 'sepal_width,3.8', 'sepal_width,3.6', 'sepal_width,3.3', 'sepal_width,4.1', 'sepal_width,2.3', 'petal_width,0.2', 'petal_width,0.4']
['sepal_width,3.5', 'sepal_width,3', 'sepal_width,3.2', 'sepal_width,3.4', 'sepal_width,2.9', 'sepal_width

['sepal_length,6.9', 'sepal_length,5.5', 'sepal_length,6.5', 'sepal_length,5.7', 'sepal_length,4.9', 'sepal_length,6.6', 'sepal_length,5.2', 'sepal_length,5.9', 'sepal_length,6.7', 'sepal_length,5.6', 'sepal_length,5.8', 'sepal_length,6.2', 'sepal_length,6.1', 'sepal_length,6.8', 'sepal_length,6', 'sepal_length,5.4', 'sepal_length,5', 'sepal_length,5.1', 'sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2', 'sepal_width,2.9', 'sepal_width,2.6', 'sepal_width,2.3', 'petal_length,4.7', 'petal_length,4.9', 'petal_length,4', 'petal_length,4.6', 'petal_length,4.5', 'petal_length,3.3', 'petal_length,3.9', 'petal_length,3.5', 'petal_length,4.2', 'petal_length,3.6', 'petal_length,4.1', 'petal_length,4.8', 'petal_length,4.3', 'petal_length,4.4', 'petal_length,5', 'petal_length,3.8', 'petal_length,3.7']
['sepal_length,6.9', 'sepal_length,5.5', 'sepal_length,6.5', 'sepal_length,5.7', 'sepal_length,4.9', 'sepal_length,6.6'

['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2', 'sepal_width,2.9']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2', 'sepal_width,2.9', 'sepal_width,2.6']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2', 'sepal_width,2.9', 'sepal_width,2.6', 'sepal_width,2.3']
Union Set:  {2, 3, 9, 10, 13, 14, 26, 30, 31, 35, 36, 39, 42, 46, 48, 51, 53, 55, 56, 58, 60, 62, 63, 64, 65, 66, 69, 71, 72, 74, 75, 78, 80, 84, 87, 89, 91, 92, 93, 94, 95, 97, 98, 100, 102, 104, 106, 111, 113, 119, 120, 122, 126, 127, 128, 129, 130, 131, 133, 135, 136, 138, 139, 140, 141, 143, 144, 146, 148}
When values are list
['sepal_width,3.2', 'sepa

Final Union Set:  [128, 139, 147, 51, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 81, 82, 83, 85, 86, 88, 90, 91, 92, 95, 96, 99, 107, 120, 122]
When item is found as key in the block
{98, 69, 149, 127}
When item is found as key in the block
{98}
Final tmp_set:  {98}
This is the final value:  set()
K_98


When value is single
['sepal_length,5.1']
When values are list
['sepal_width,3.2']
['sepal_width,3.2', 'sepal_width,3.1']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3']
['sepal_width,3.2', 'sepal_width,3.1', 'sepal_width,2.8', 'sepal_width,2.4', 'sepal_width,2.7', 'sepal_width,3', 'sepal_width,2.2']
['sepal_width,3.2', 'sepal_width,3.1', 'sep

['sepal_width,3.3', 'sepal_width,2.7', 'sepal_width,2.9', 'sepal_width,3', 'sepal_width,2.5', 'sepal_width,3.6', 'sepal_width,3.2', 'sepal_width,3.8', 'sepal_width,2.6', 'sepal_width,2.2', 'sepal_width,2.8']
['sepal_width,3.3', 'sepal_width,2.7', 'sepal_width,2.9', 'sepal_width,3', 'sepal_width,2.5', 'sepal_width,3.6', 'sepal_width,3.2', 'sepal_width,3.8', 'sepal_width,2.6', 'sepal_width,2.2', 'sepal_width,2.8', 'sepal_width,3.4']
['sepal_width,3.3', 'sepal_width,2.7', 'sepal_width,2.9', 'sepal_width,3', 'sepal_width,2.5', 'sepal_width,3.6', 'sepal_width,3.2', 'sepal_width,3.8', 'sepal_width,2.6', 'sepal_width,2.2', 'sepal_width,2.8', 'sepal_width,3.4', 'sepal_width,3.1']
Union Set:  {2, 3, 7, 8, 9, 10, 12, 13, 14, 19, 20, 23, 24, 25, 26, 27, 29, 30, 31, 35, 36, 39, 46, 48, 51, 53, 55, 56, 60, 62, 63, 64, 65, 66, 69, 71, 72, 74, 75, 78, 80, 84, 87, 89, 91, 92, 93, 95, 97, 98, 100, 101, 102, 104, 106, 107, 109, 110, 111, 113, 114, 118, 119, 120, 122, 126, 127, 128, 129, 130, 131, 132, 1

['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,4.9', 'sepal_length,7.3', 'sepal_length,7.2', 'sepal_length,6.8', 'sepal_length,7.7', 'sepal_length,6', 'sepal_length,6.9', 'sepal_length,5.6']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,4.9', 'sepal_length,7.3', 'sepal_length,7.2', 'sepal_length,6.8', 'sepal_length,7.7', 'sepal_length,6', 'sepal_length,6.9', 'sepal_length,5.6', 'sepal_length,6.2']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,4.9', 'sepal_length,7.3', 'sepal_length,7.2', 'sepal_length,6.8', 'sepal_length,7.7', 'sepal_length,6', 'sepal_length,6.9', 'sepal_length,5.6', 'sepal_length,6.2', 'sepal_length,6.1']
['sepal_length,6.3', 'sepal_length,5.8', 'sepal_length,7.1', 'sepal_length,6.5', 'sepal_length,4.9', 'sepal_length,7.3', 'sepal_length,7.2', 'sepal_length,6.8', 'sepal_length,7.7', 'sepal_length,6', 'sepal_length,6.9', 'sep

In [40]:
dic

{'K_1': {18},
 'K_2': {13, 14, 26, 46},
 'K_3': set(),
 'K_4': set(),
 'K_5': {50},
 'K_6': {17, 22},
 'K_7': set(),
 'K_8': set(),
 'K_9': set(),
 'K_10': set(),
 'K_11': {11},
 'K_12': {25},
 'K_13': {46},
 'K_14': set(),
 'K_15': set(),
 'K_16': {17, 22},
 'K_17': set(),
 'K_18': set(),
 'K_19': set(),
 'K_20': set(),
 'K_21': {11},
 'K_22': set(),
 'K_23': set(),
 'K_24': {24},
 'K_25': {25},
 'K_26': {26},
 'K_27': set(),
 'K_28': set(),
 'K_29': {7},
 'K_30': {3},
 'K_31': set(),
 'K_32': {11},
 'K_33': set(),
 'K_34': {7, 9, 13, 18, 46, 48, 50},
 'K_35': set(),
 'K_36': set(),
 'K_37': set(),
 'K_38': set(),
 'K_39': {26},
 'K_40': set(),
 'K_41': set(),
 'K_42': {94},
 'K_43': set(),
 'K_44': {44},
 'K_45': {17, 22},
 'K_46': {46},
 'K_47': {26},
 'K_48': set(),
 'K_49': set(),
 'K_50': set(),
 'K_51': set(),
 'K_52': {53, 62, 69, 120},
 'K_53': {53},
 'K_54': set(),
 'K_55': set(),
 'K_56': {56},
 'K_57': set(),
 'K_58': {58},
 'K_59': set(),
 'K_60': set(),
 'K_61': set(),
 '

#### Lower Approximation !!

In [41]:
def lowerApproximation(charac_set,concept):
    #set to contain lower approximations
    lower = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
  
        if set_value.issubset(set(concept)):
            lower = lower.union(set_value)
          
    return lower

In [42]:
lower_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    lower_approximations[str(item)] = lowerApproximation(dic,item)

In [43]:
lower_goal_list=list(lower_approximations.values())
print(lower_goal_list)

[{3, 7, 9, 11, 13, 14, 17, 18, 22, 24, 25, 26, 44, 46, 48, 50}, {69, 71, 72, 53, 54, 56, 58, 92, 62}, {128, 135, 136, 137, 144, 146, 147, 148, 101, 102, 104, 107, 118, 120, 121, 122}]


#### Upper Approximation !!

In [44]:
def upperApproximation(charac_set,concept):
    #set to contain lower approximations
    upper = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
        upper = upper.union(set_value)
          
    return upper

In [45]:
upper_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    upper_approximations[str(item)] = upperApproximation(dic,item)

In [46]:
upper_goal_list=list(upper_approximations.values())
print(upper_goal_list)

[{3, 7, 9, 11, 13, 14, 17, 18, 22, 24, 25, 26, 94, 44, 46, 48, 50}, {56, 69, 71, 72, 53, 54, 120, 58, 92, 62}, {128, 71, 135, 136, 137, 144, 146, 147, 148, 62, 101, 102, 104, 107, 108, 118, 55, 120, 121, 122, 126, 127}]


#### Middle Approximation !!

In [47]:
first_column = list(dic.keys())

In [48]:
second_column = list(dic.values())

In [49]:
prob_approx = pd.DataFrame(
    {'charset_name': first_column,
     'charset_value': second_column
    })

In [50]:
prob_approx

,charset_name,charset_value
0,K_1,{18}
1,K_2,"{46, 26, 13, 14}"
2,K_3,{}
3,K_4,{}
4,K_5,{50}
5,K_6,"{17, 22}"
6,K_7,{}
7,K_8,{}
8,K_9,{}
9,K_10,{}


In [51]:
def probabilisticApproximation(concept):
    prob = []
    for index, row in prob_approx.iterrows():
        if len(row['charset_value']) != 0:
            probability_conditional = len(row['charset_value'].intersection(set(concept))) / len(row['charset_value'])
        else:
            probability_conditional = 0.0
            
        prob.append(round(probability_conditional,2))
    return prob

In [52]:
def findMiddleApprox(concept):
    probapprox = set()
    cond_prob = probabilisticApproximation(concept) #Need to put the goal
    prob_approx['cond_probability'] = cond_prob
    print(prob_approx)
    for index, row in prob_approx.iterrows():
        part1,part2 = row['charset_name'].split("_")
       
        if row['cond_probability'] >= 0.50:
            if int(part2) in concept:
#                 print(row['charset_value'])
                probapprox = probapprox.union(row['charset_value'])
    
    return probapprox  

In [53]:
middle_approximations = {}
for item in goal_list:
    print(item)
    #Key is the string converted list so as to add as dictionary key
    middle_approximations[str(item)] = findMiddleApprox(item)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
    charset_name                  charset_value  cond_probability
0            K_1                           {18}               1.0
1            K_2               {46, 26, 13, 14}               1.0
2            K_3                             {}               0.0
3            K_4                             {}               0.0
4            K_5                           {50}               1.0
5            K_6                       {17, 22}               1.0
6            K_7                             {}               0.0
7            K_8                             {}               0.0
8            K_9                             {}               0.0
9           K_10                             {}               0.0
10          K_11                           {11}               1.0
11          K_12

In [54]:
middle_goal_list=list(middle_approximations.values())
print(middle_goal_list)

[{3, 7, 9, 11, 13, 14, 17, 18, 22, 24, 25, 26, 44, 46, 48, 50}, {56, 69, 71, 72, 53, 54, 120, 58, 92, 62}, {128, 71, 135, 136, 137, 144, 146, 147, 148, 101, 102, 104, 107, 108, 118, 55, 120, 121, 122, 126, 127}]


In [55]:
df2

,Cases,att_val
0,"sepal_length,4.3..4.35","[2, 34, 35, 5, 6, 36, 8, 39, 41, 42, 12, 43, 1..."
1,"sepal_length,4.35..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15..."
2,"sepal_length,4.3..4.5","[2, 5, 6, 8, 9, 12, 14, 16, 19, 23, 27, 29, 34..."
3,"sepal_length,4.5..7.9","[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1..."
4,"sepal_length,4.3..4.65","[2, 4, 5, 6, 7, 8, 9, 12, 14, 16, 19, 23, 27, ..."
5,"sepal_length,4.65..7.9","[1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17,..."
6,"sepal_length,4.3..4.75","[2, 3, 4, 5, 6, 7, 8, 9, 12, 14, 16, 19, 23, 2..."
7,"sepal_length,4.75..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 13, 15, 16, 17, 18..."
8,"sepal_length,4.3..4.85","[2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 16, 19, 2..."
9,"sepal_length,4.85..7.9","[1, 2, 5, 6, 8, 10, 11, 12, 15, 16, 17, 18, 19..."


In [56]:
df3=df2.copy()

In [57]:
import pylab as pl

In [58]:
def findGoalIntersect(goal):
    goalIntersect = []
    
    for index, row in df3.iterrows():
        #List containing intersection of (a,v) pairs and goal
        goalIntersect.append(set(row['att_val']).intersection(set(goal)))
          
    #Check if goal_intersect column exists
    if 'goal_intersect' in df3:
        df3['goal_intersect'] = goalIntersect
    else:
        #Insert new column with the recent iteration
        df3.insert(2, 'goal_intersect', goalIntersect)

In [59]:
def findCases(df3):
    case_to_be = []
    #Find the cases with maximum goal coverage
    m = max(df3['goal_intersect'], key=len)
    possible_cases = [i for i, j in enumerate(df3['goal_intersect'].tolist()) if j == m]
    
    #Index of the case covering max goal and having min no. of elements
    new_df = df3.iloc[possible_cases,:]
    
    m1 = min(new_df['att_val'], key=len)
    
    for index,row in new_df.iterrows():
        if row['att_val'] == m1:
            case_to_be.append(index)
   
    return case_to_be[0]

In [60]:
def combineInterval(test_condition):
    
    test_num = [] #This will contain the conditions having interval
    test_str = [] #This will contain the conditions having no interval
    
    #Loop through to seprate contions having intervals and no intervals
    for item in test_condition:
        if ".." in item:
            test_num.append(item)
        else:
            test_str.append(item)
   
    #Group the conditions having interval based on same attributes
    grouped = [list(g) for k, g in groupby(test_num, lambda s: s.partition(',')[0])]
    
    final_list = []
    
    #Actually combining the intervals
    for list1 in grouped:
        greatest = 0
        smallest = 0
        for item in list1:
            part1,part2 = item.split(",")
            start,stop = part2.split("..")
            start = float(start)
            stop = float(stop)

            if greatest == 0 and smallest == 0:
                greatest = start
                smallest = stop

            if start > greatest:
                greatest = float(start)

            if stop < smallest:
                smallest = float(stop)

        con_tmp = part1 + "," + str(greatest) + ".." + str(smallest)
        final_list.append(con_tmp)  
            
    actual_condition = final_list + test_str
    
    return actual_condition

In [61]:
def dropCondition(condition,current_goal):
    
    for item in range(0,len(condition)):
        temp_att_val = []
        temp_cond = condition.copy() #use list.copy() as equal operator simply copies over the reference
        temp_cond.remove(condition[item])
       
        #temp_cond contains the elements after removing the current element
        for i in temp_cond:
            if i is not None:
                location = df3.index[df3['Cases'] == i].tolist()
                element = df3['att_val'].loc[location[0]]
                temp_att_val.append(set(element))
          
        #temp_att_val contains the actual value set of the corresponding cases
        #Find the intersection if the set has more than one element, otherwise no need
        if len(temp_att_val) > 1:
            intersection = set.intersection(*temp_att_val)
        
            #if the set still remains a subset of the original goal after removing current element
            #then set the current element to None as we want to drop this later
            if intersection.issubset(current_goal):
                condition[item] = None
    
    condition = [x for x in condition if x is not None]
   
    return condition   

In [62]:
def stepAlgo(df3,selected_case,current_goal,B,condition,concept_curr):
    
    rule_set = []
    original_goal = current_goal.copy()
    
    
    while current_goal != set():
        #Check if the selected case is a subset of the current goal
        #List of current case
        A = df3['att_val'].loc[selected_case] 

        if B == set():
            #Copy over the current set elements to B
            for i in range(len(A)):
                B.add(A[i])

        #Elements of intersection of current and previous set
        A = set(A).intersection(B)
        B = A.copy()
        
        print("Intersection set : ",A)
        #Check if intersecting elements are subset of Goal
        if A.issubset(current_goal):
          
            #Current goal is updated after discarding the already covered goal by new rule
            if len(A) != 0:
                current_goal = set(current_goal) - A
            else:
                current_goal = set()
            

            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the conditions of a Rule
            condition.append(curr_case)
            
            #Check for possibility of dropping conditions
            if len(condition) > 1:
                condition = dropCondition(condition,original_goal)
            
            #Combine the interval
            if len(condition) > 1:
                condition = combineInterval(condition)
            

            #Join conditions
            cond = ""
            for item in condition:
                cond = cond + "(" + str(item) + ")" + " & "

            cond = cond[:-2] + "->"
            rule = cond + " (" + concept + "," + concept_curr + ")"
            rule_set.append(rule)
            
            #Reset everythng and continue for covering rest of the goal
            condition = []
            B = set()
            findGoalIntersect(current_goal)
            selected_case = findCases(df3)


        #If not a subset of current goal
        else:
            print("NOT")
            #Assign empty set for the selected case for next iteration
            df3['goal_intersect'].loc[selected_case] = set()
            print(df3['Cases'].loc[selected_case], " Set to be NULL")
           
            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the case to the condition list
            condition.append(curr_case)
            
            #Check for Range overlapping of the remaining cases
            if ".." in curr_case:
                second_part = curr_case.split(',')[1]
                start = float(second_part.split('..')[0])
                end = float(second_part.split('..')[1])
                for index, row in df3.iterrows():
                    if ".." in row['Cases']:
                        part2 = row['Cases'].split(',')[1]
                        start1 = float(part2.split('..')[0])
                        end1 = float(part2.split('..')[1])

                        #Assign blank set for cases with overlapping ranges
                        if set((pl.frange(start,end))).issubset(pl.frange(start1,end1)) == True:
                            if start == start1 and end <= end1:
                                row['goal_intersect'] = set()
                                print(row['Cases'], "also set to be NULL")
            
           
            selected_case = findCases(df3)
           
                        
    return rule_set

In [63]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(middle_goal_list)):
    findGoalIntersect(list(middle_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,middle_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
petal_length,1.0..2.45  Set to be NULL
petal_length,1.0..2.45 also set to be NULL
petal_length,1.0..3.15 also set to be NULL
petal_length,1.0..3.4 also set to be NULL
petal_length,1.0..3.55 also set to be NULL
petal_length,1.0..3.65 also set to be NULL
petal_length,1.0..3.75 also set to be NULL
petal_length,1.0..3.85 also set to be NULL
petal_length,1.0..3.95 also set to be NULL
petal_length,1.0..4.05 also set to be NULL
petal_length,1.0..4.15 also set to be NULL
petal_length,1.0..4.25 also set to be NULL
petal_length,1.0..4.35 also set to be NULL
petal_length,1.0..4.45 also set to be NULL
petal_length,1.0..4.55 also set to be NULL
petal_length,1.0..4.65 also set to be NULL
petal_length,1.0..4.75 also set to be NULL
petal_length,1.0..4.85 also set to be NULL
petal_length,1.

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


sepal_length,4.3..7.15 also set to be NULL
sepal_length,4.3..7.25 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.85..4.1  Set to be NULL
sepal_width,2.85..4.1 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.75..4.1  Set to be NULL
sepal_width,2.75..4.1 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.65..4.1  Set to be NULL
sepal_width,2.65..4.

petal_width,0.1..1.35 also set to be NULL
petal_width,0.1..1.45 also set to be NULL
petal_width,0.1..1.55 also set to be NULL
petal_width,0.1..1.65 also set to be NULL
petal_width,0.1..1.75 also set to be NULL
petal_width,0.1..1.85 also set to be NULL
petal_width,0.1..1.95 also set to be NULL
petal_width,0.1..2.05 also set to be NULL
petal_width,0.1..2.15 also set to be NULL
petal_width,0.1..2.25 also set to be NULL
petal_width,0.1..2.35 also set to be NULL
petal_width,0.1..2.45 also set to be NULL
Intersection set :  {1, 2, 5, 6, 8, 10, 12, 13, 16, 18, 26, 27, 29, 30, 31, 34, 39, 40, 45, 46, 47, 50}
NOT
sepal_length,4.65..7.9  Set to be NULL
sepal_length,4.65..7.9 also set to be NULL
Intersection set :  {2, 34, 5, 6, 39, 8, 40, 10, 12, 13, 45, 46, 16, 47, 50, 26, 27, 29, 30, 31}
NOT
sepal_width,2.2..3.45  Set to be NULL
sepal_width,2.2..3.45 also set to be NULL
sepal_width,2.2..3.55 also set to be NULL
sepal_width,2.2..3.65 also set to be NULL
sepal_width,2.2..3.75 also set to be NULL

Intersection set :  {34}
NOT
sepal_width,3.25..4.1  Set to be NULL
sepal_width,3.25..4.1 also set to be NULL
Intersection set :  {34}
NOT
sepal_length,4.3..4.85  Set to be NULL
sepal_length,4.3..4.85 also set to be NULL
sepal_length,4.3..4.95 also set to be NULL
sepal_length,4.3..5.05 also set to be NULL
sepal_length,4.3..5.15 also set to be NULL
sepal_length,4.3..5.25 also set to be NULL
sepal_length,4.3..5.35 also set to be NULL
sepal_length,4.3..5.45 also set to be NULL
sepal_length,4.3..5.55 also set to be NULL
sepal_length,4.3..5.65 also set to be NULL
sepal_length,4.3..5.75 also set to be NULL
sepal_length,4.3..5.85 also set to be NULL
sepal_length,4.3..5.95 also set to be NULL
sepal_length,4.3..6.05 also set to be NULL
sepal_length,4.3..6.15 also set to be NULL
sepal_length,4.3..6.25 also set to be NULL
sepal_length,4.3..6.35 also set to be NULL
sepal_length,4.3..6.45 also set to be NULL
sepal_length,4.3..6.55 also set to be NULL
sepal_length,4.3..6.65 also set to be NULL
sepal_

...

petal_length,4.45..6.7 also set to be NULL
Intersection set :  {52, 150, 79}
NOT
petal_length,4.35..6.7  Set to be NULL
sepal_length,4.35..7.9 also set to be NULL
petal_length,4.35..6.7 also set to be NULL
Intersection set :  {52, 150, 79}
NOT
petal_length,4.25..6.7  Set to be NULL
petal_length,4.25..6.7 also set to be NULL
Intersection set :  {52, 150, 79}
NOT
petal_length,1.0..4.55  Set to be NULL
petal_length,1.0..4.55 also set to be NULL
petal_length,1.0..4.65 also set to be NULL
petal_length,1.0..4.75 also set to be NULL
petal_length,1.0..4.85 also set to be NULL
petal_length,1.0..4.95 also set to be NULL
petal_length,1.0..5.05 also set to be NULL
petal_length,1.0..5.15 also set to be NULL
petal_length,1.0..5.25 also set to be NULL
petal_length,1.0..5.4 also set to be NULL
petal_length,1.0..5.55 also set to be NULL
petal_length,1.0..5.65 also set to be NULL
petal_length,1.0..5.75 also set to be NULL
petal_length,1.0..5.85 also set to be NULL
petal_length,1.0..5.95 also set to be N

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,3.15..6.7  Set to be NULL
petal_length,3.15..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,2.45..6.7  Set to be NULL
petal_length,2.45..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 

petal_length,1.1..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 71, 73, 77, 78, 84, 87, 92, 98, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,4.55..6.7  Set to be NULL
petal_length,4.55..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 71, 73, 77, 78, 84, 87, 92, 98, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
sepal_length,5.55..7.9  Set to be NULL
sepal_length,5.55..7.9 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51,

Intersection set :  {128, 129, 130, 131, 133, 134, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 52, 53, 71, 73, 78, 84, 87, 98, 102, 103, 104, 105, 108, 111, 112, 113, 115, 116, 117, 121, 124, 125, 126, 127}
NOT
petal_length,4.75..6.7  Set to be NULL
sepal_length,4.75..7.9 also set to be NULL
petal_length,4.75..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 52, 53, 71, 73, 78, 84, 87, 98, 102, 103, 104, 105, 108, 111, 112, 113, 115, 116, 117, 121, 124, 125, 126, 127}
NOT
petal_length,4.65..6.7  Set to be NULL
sepal_length,4.65..7.9 also set to be NULL
petal_length,4.65..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 52, 53, 71, 73, 78, 84, 87, 98, 102, 103, 104, 105, 111, 112, 113, 115, 116, 117, 121, 124, 125, 126, 127}
NOT
petal_length,1.0..6.2  Set to be NULL
petal_length,1.0..6.2 also set to be

petal_width,0.1..2.25 also set to be NULL
petal_width,0.1..2.35 also set to be NULL
petal_width,0.1..2.45 also set to be NULL
Intersection set :  {129, 98, 133, 104, 105, 78, 112, 147, 148, 115, 117, 124, 127}
NOT
sepal_length,6.15..7.9  Set to be NULL
sepal_length,6.15..7.9 also set to be NULL
Intersection set :  {129, 98, 133, 104, 105, 112, 147, 148, 115, 117, 124, 127}
NOT
sepal_length,4.3..6.55  Set to be NULL
sepal_length,4.3..6.55 also set to be NULL
sepal_length,4.3..6.65 also set to be NULL
sepal_length,4.3..6.75 also set to be NULL
sepal_length,4.3..6.85 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.15 also set to be NULL
sepal_length,4.3..7.25 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
Intersection set :  {98, 104, 105, 112, 147, 148, 115, 117, 124}
NOT
sepal_width,2.85..4.1  Set to be NULL
sepal_width,2.85..4.1 also set to be NULL
Intersection set :  {104, 105, 112, 115, 148, 11

In [64]:
print("Time to run the algorithm (for Middle Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_35_attcon_middle.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Middle Approximation):  15.988 Sec


['(sepal_length,4.85..4.85) & (sepal_width,3.35..4.1) & (petal_width,0.45..2.5) & (sepal_width,2.2..3.05) & (petal_length,1.45..6.7) -> (class,Iris-setosa)']
['(petal_length,4.25..4.55) & (petal_width,1.45..2.5) & (sepal_length,5.95..5.95) & (sepal_width,2.95..4.1) & (petal_width,0.1..1.45) -> (class,Iris-versicolor)']
['(sepal_length,6.25..7.9) & (sepal_width,2.95..4.1) & (sepal_length,4.3..6.35) & (petal_length,5.25..6.7) & (petal_width,1.85..2.5) & (petal_length,1.0..5.15) & (petal_width,0.1..1.85) -> (class,Iris-viginica)']


In [65]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(lower_goal_list)):
    findGoalIntersect(list(lower_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,lower_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
petal_length,1.0..2.45  Set to be NULL
petal_length,1.0..2.45 also set to be NULL
petal_length,1.0..3.15 also set to be NULL
petal_length,1.0..3.4 also set to be NULL
petal_length,1.0..3.55 also set to be NULL
petal_length,1.0..3.65 also set to be NULL
petal_length,1.0..3.75 also set to be NULL
petal_length,1.0..3.85 also set to be NULL
petal_length,1.0..3.95 also set to be NULL
petal_length,1.0..4.05 also set to be NULL
petal_length,1.0..4.15 also set to be NULL
petal_length,1.0..4.25 also set to be NULL
petal_length,1.0..4.35 also set to be NULL
petal_length,1.0..4.45 also set to be NULL
petal_length,1.0..4.55 also set to be NULL
petal_length,1.0..4.65 also set to be NULL
petal_length,1.0..4.75 also set to be NULL
petal_length,1.0..4.85 also set to be NULL
petal_length,1.

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.65..4.1  Set to be NULL
sepal_width,2.65..4.1 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.55..4.1  Set to be NULL
sepal_width,2.55..4.1 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50}
NOT
sepal_width,2.45..4.1  Set to be NULL
sepal_width,2.45..4.1 also set to be NULL
petal_length,2.45..6.7 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2

petal_length,1.0..2.45 also set to be NULL
petal_length,1.0..3.15 also set to be NULL
petal_length,1.0..3.4 also set to be NULL
petal_length,1.0..3.55 also set to be NULL
petal_length,1.0..3.65 also set to be NULL
petal_length,1.0..3.75 also set to be NULL
petal_length,1.0..3.85 also set to be NULL
petal_length,1.0..3.95 also set to be NULL
petal_length,1.0..4.05 also set to be NULL
petal_length,1.0..4.15 also set to be NULL
petal_length,1.0..4.25 also set to be NULL
petal_length,1.0..4.35 also set to be NULL
petal_length,1.0..4.45 also set to be NULL
petal_length,1.0..4.55 also set to be NULL
petal_length,1.0..4.65 also set to be NULL
petal_length,1.0..4.75 also set to be NULL
petal_length,1.0..4.85 also set to be NULL
petal_length,1.0..4.95 also set to be NULL
petal_length,1.0..5.05 also set to be NULL
petal_length,1.0..5.15 also set to be NULL
petal_length,1.0..5.25 also set to be NULL
petal_length,1.0..5.4 also set to be NULL
petal_length,1.0..5.55 also set to be NULL
petal_length,

Intersection set :  {34}
NOT
sepal_width,3.35..4.1  Set to be NULL
sepal_width,3.35..4.1 also set to be NULL
Intersection set :  {34}
NOT
petal_width,0.45..2.5  Set to be NULL
petal_width,0.45..2.5 also set to be NULL
Intersection set :  {34}
NOT
sepal_width,2.2..3.05  Set to be NULL
sepal_width,2.2..3.05 also set to be NULL
sepal_width,2.2..3.15 also set to be NULL
sepal_width,2.2..3.25 also set to be NULL
sepal_width,2.2..3.35 also set to be NULL
sepal_width,2.2..3.45 also set to be NULL
sepal_width,2.2..3.55 also set to be NULL
sepal_width,2.2..3.65 also set to be NULL
sepal_width,2.2..3.75 also set to be NULL
sepal_width,2.2..3.95 also set to be NULL
Intersection set :  set()
Intersection set :  {51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119

...

Intersection set :  {150, 51, 52, 54, 56, 57, 62, 64, 67, 68, 72, 75, 76, 79, 84, 88, 89, 90, 91, 92, 95, 96, 97, 98, 100, 114}
NOT
petal_width,0.1..1.55  Set to be NULL
petal_width,0.1..1.55 also set to be NULL
petal_width,0.1..1.65 also set to be NULL
petal_width,0.1..1.75 also set to be NULL
petal_width,0.1..1.85 also set to be NULL
petal_width,0.1..1.95 also set to be NULL
petal_width,0.1..2.05 also set to be NULL
petal_width,0.1..2.15 also set to be NULL
petal_width,0.1..2.25 also set to be NULL
petal_width,0.1..2.35 also set to be NULL
petal_width,0.1..2.45 also set to be NULL
Intersection set :  {150, 52, 54, 56, 57, 62, 64, 67, 68, 72, 75, 76, 79, 84, 88, 89, 90, 91, 92, 95, 96, 97, 98, 100, 114}
NOT
sepal_width,2.2..3.15  Set to be NULL
sepal_width,2.2..3.15 also set to be NULL
sepal_width,2.2..3.25 also set to be NULL
sepal_width,2.2..3.35 also set to be NULL
sepal_width,2.2..3.45 also set to be NULL
sepal_width,2.2..3.55 also set to be NULL
sepal_width,2.2..3.65 also set to 

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 56, 57, 59, 64, 66, 67, 69, 71, 73, 74, 77, 78, 79, 80, 84, 85, 86, 87, 89, 92, 93, 94, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,4.45..6.7  Set to be NULL
petal_length,4.45..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 56, 57, 59, 64, 66, 67, 69, 71, 73, 74, 77, 78, 79, 80, 84, 85, 86, 87, 89, 92, 93, 94, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,4.35..6.7  Set to be NULL
sepal_length,4.35..7.9 also set to be NULL
petal_length,4.35..6.7 also set to be NULL
Intersect

petal_width,0.45..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_width,0.35..2.5  Set to be NULL
petal_width,0.35..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 67, 69, 71, 73, 77, 78, 79, 84, 85, 86, 87, 92, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_width,0.25..2.5  Set to be NULL
petal_width,0.25..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 13

sepal_width,2.45..4.1 also set to be NULL
petal_length,2.45..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 52, 53, 66, 73, 77, 78, 84, 87, 98, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127}
NOT
sepal_width,2.35..4.1  Set to be NULL
sepal_width,2.35..4.1 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 52, 53, 66, 73, 77, 78, 84, 87, 98, 100, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127}
NOT
sepal_width,2.25..4.1  Set to be NULL
sepal_width,2.25..4.1 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 52, 53, 66, 73, 77, 78, 84, 87, 98, 1

Intersection set :  {129, 98, 133, 104, 105, 142, 112, 145, 147, 148, 115, 116, 117, 124, 125, 127}
NOT
sepal_length,4.3..6.55  Set to be NULL
sepal_length,4.3..6.55 also set to be NULL
sepal_length,4.3..6.65 also set to be NULL
sepal_length,4.3..6.75 also set to be NULL
sepal_length,4.3..6.85 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.15 also set to be NULL
sepal_length,4.3..7.25 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
Intersection set :  {98, 104, 105, 142, 112, 145, 147, 148, 115, 116, 117, 124, 125}
NOT
sepal_width,2.85..4.1  Set to be NULL
sepal_width,2.85..4.1 also set to be NULL
Intersection set :  {104, 105, 142, 112, 145, 115, 148, 116, 117, 124, 125}
NOT
petal_length,5.05..6.7  Set to be NULL
sepal_length,5.05..7.9 also set to be NULL
petal_length,5.05..6.7 also set to be NULL
Intersection set :  {104, 105, 142, 112, 115, 148, 116, 117, 124}
NOT
petal_length,1.0..5.65  Set t

In [66]:
print("Time to run the algorithm (for Lower Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_35_attcon_lower.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Lower Approximation):  11.94 Sec


['(sepal_length,4.85..4.85) & (sepal_width,3.35..4.1) & (petal_width,0.45..2.5) & (sepal_width,2.2..3.05) & (petal_length,1.45..6.7) -> (class,Iris-setosa)']
['(petal_length,1.0..4.55) & (sepal_length,4.3..5.95) & (sepal_width,2.85..4.1) & (petal_length,4.35..6.7) & (petal_width,1.45..1.45) & (sepal_length,6.05..7.9) -> (class,Iris-versicolor)']
['(petal_length,1.0..5.65) & (sepal_length,6.25..6.35) & (sepal_width,2.95..4.1) & (petal_length,5.15..6.7) & (petal_width,1.95..2.25) -> (class,Iris-viginica)']


In [67]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(upper_goal_list)):
    findGoalIntersect(list(upper_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,upper_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    
elapsed_time = time.time() - start_time

Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 59, 66, 77, 80, 84, 87, 89, 93, 94, 97, 98, 99, 100}
NOT
petal_length,1.0..3.15  Set to be NULL
petal_length,1.0..3.15 also set to be NULL
petal_length,1.0..3.4 also set to be NULL
petal_length,1.0..3.55 also set to be NULL
petal_length,1.0..3.65 also set to be NULL
petal_length,1.0..3.75 also set to be NULL
petal_length,1.0..3.85 also set to be NULL
petal_length,1.0..3.95 also set to be NULL
petal_length,1.0..4.05 also set to be NULL
petal_length,1.0..4.15 also set to be NULL
petal_length,1.0..4.25 also set to be NULL
petal_length,1.0..4.35 also set to be NULL
petal_length,1.0..4.45 also set to be NULL
petal_length,1.0..4.55 also set to be NULL
petal_length,1.0..4.65 also set to be NULL
petal_length,1.0..4.75 also set to be NULL
petal_length,1.0..4.85 also set to be NULL
p

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: MatplotlibDeprecationWarning: numpy.arange


 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 94, 99}
NOT
petal_length,1.25..6.7  Set to be NULL
petal_length,1.25..6.7 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 94, 99}
NOT
petal_length,1.1..6.7  Set to be NULL
petal_length,1.1..6.7 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 94, 99}
NOT
petal_width,0.15..2.5  Set to be NULL
petal_width,0.15..2.5 also set to be NULL
Intersection set :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4

Intersection set :  {2, 34, 5, 6, 39, 8, 40, 10, 12, 13, 45, 46, 16, 47, 50, 26, 27, 29, 30, 31}
NOT
sepal_width,2.2..3.45  Set to be NULL
sepal_width,2.2..3.45 also set to be NULL
sepal_width,2.2..3.55 also set to be NULL
sepal_width,2.2..3.65 also set to be NULL
sepal_width,2.2..3.75 also set to be NULL
sepal_width,2.2..3.95 also set to be NULL
Intersection set :  {2, 34, 5, 6, 39, 8, 40, 10, 12, 13, 45, 46, 16, 47, 50, 26, 27, 29, 31}
NOT
sepal_length,4.75..7.9  Set to be NULL
sepal_length,4.75..7.9 also set to be NULL
Intersection set :  {2, 34, 5, 6, 39, 8, 10, 12, 13, 45, 46, 16, 47, 50, 26, 27, 29, 31}
NOT
sepal_length,4.3..5.05  Set to be NULL
sepal_length,4.3..5.05 also set to be NULL
sepal_length,4.3..5.15 also set to be NULL
sepal_length,4.3..5.25 also set to be NULL
sepal_length,4.3..5.35 also set to be NULL
sepal_length,4.3..5.45 also set to be NULL
sepal_length,4.3..5.55 also set to be NULL
sepal_length,4.3..5.65 also set to be NULL
sepal_length,4.3..5.75 also set to be N

sepal_length,4.3..5.15 also set to be NULL
sepal_length,4.3..5.25 also set to be NULL
sepal_length,4.3..5.35 also set to be NULL
sepal_length,4.3..5.45 also set to be NULL
sepal_length,4.3..5.55 also set to be NULL
sepal_length,4.3..5.65 also set to be NULL
sepal_length,4.3..5.75 also set to be NULL
sepal_length,4.3..5.85 also set to be NULL
sepal_length,4.3..5.95 also set to be NULL
sepal_length,4.3..6.05 also set to be NULL
sepal_length,4.3..6.15 also set to be NULL
sepal_length,4.3..6.25 also set to be NULL
sepal_length,4.3..6.35 also set to be NULL
sepal_length,4.3..6.45 also set to be NULL
sepal_length,4.3..6.55 also set to be NULL
sepal_length,4.3..6.65 also set to be NULL
sepal_length,4.3..6.75 also set to be NULL
sepal_length,4.3..6.85 also set to be NULL
sepal_length,4.3..7.0 also set to be NULL
sepal_length,4.3..7.15 also set to be NULL
sepal_length,4.3..7.25 also set to be NULL
sepal_length,4.3..7.5 also set to be NULL
sepal_length,4.3..7.8 also set to be NULL
Intersection s

...

Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 54, 55, 57, 60, 61, 62, 64, 66, 67, 68, 69, 71, 73, 75, 76, 77, 78, 79, 83, 84, 85, 86, 87, 90, 91, 92, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_width,1.35..2.5  Set to be NULL
petal_length,1.35..6.7 also set to be NULL
petal_width,1.35..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 62, 64, 66, 67, 69, 71, 73, 75, 76, 77, 78, 79, 84, 85, 86, 87, 91, 92, 95, 96, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,4.15..6.7  Set to be NULL
petal_length,4.15..6.7 also set to be NULL

petal_width,0.25..2.5 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 62, 64, 66, 67, 69, 71, 73, 75, 76, 77, 78, 79, 84, 85, 86, 87, 91, 92, 95, 96, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,1.45..6.7  Set to be NULL
petal_length,1.45..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 62, 64, 66, 67, 69, 71, 73, 75, 76, 77, 78, 79, 84, 85, 86, 87, 91, 92, 95, 96, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}
NOT
sepal_length,4.85..7.9  Set to be NULL
sepal_length,4.85..7.9 also set to be NULL
Intersection set :  {128, 

petal_length,4.35..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 67, 71, 73, 77, 78, 79, 84, 86, 87, 92, 98, 100, 101, 102, 103, 104, 105, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 119, 121, 122, 123, 124, 125, 126, 127}
NOT
petal_length,4.25..6.7  Set to be NULL
petal_length,4.25..6.7 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 51, 52, 53, 55, 57, 64, 66, 67, 71, 73, 77, 78, 79, 84, 86, 87, 92, 98, 100, 101, 102, 103, 104, 105, 108, 109, 111, 112, 113, 114, 115, 116, 117, 119, 121, 122, 124, 125, 126, 127}
NOT
petal_length,1.0..6.35  Set to be NULL
petal_length,1.0..6.35 also set to be NULL
petal_length,1.0..6.5 also set to be NULL
petal_length,1.0..6.65 also set to be NULL
Intersection set :  {128, 129, 130, 131, 133, 134, 136, 137, 138

sepal_length,4.85..7.9 also set to be NULL
petal_length,4.85..6.7 also set to be NULL
Intersection set :  {128, 129, 133, 138, 140, 141, 142, 144, 145, 146, 147, 148, 150, 78, 87, 98, 104, 105, 111, 112, 113, 115, 116, 117, 121, 124, 125, 127}
NOT
petal_length,1.0..5.95  Set to be NULL
petal_length,1.0..5.95 also set to be NULL
petal_length,1.0..6.05 also set to be NULL
petal_length,1.0..6.2 also set to be NULL
petal_length,1.0..6.35 also set to be NULL
petal_length,1.0..6.5 also set to be NULL
petal_length,1.0..6.65 also set to be NULL
Intersection set :  {128, 129, 133, 138, 140, 141, 142, 144, 145, 146, 147, 148, 150, 78, 87, 98, 104, 105, 111, 112, 113, 115, 116, 117, 121, 124, 125, 127}
NOT
petal_width,1.75..2.5  Set to be NULL
petal_width,1.75..2.5 also set to be NULL
Intersection set :  {128, 129, 133, 138, 140, 141, 142, 144, 145, 146, 147, 148, 78, 87, 98, 104, 105, 111, 112, 113, 115, 116, 117, 121, 124, 125, 127}
NOT
sepal_length,5.95..7.9  Set to be NULL
sepal_length,5.95..

Intersection set :  {112, 124}
NOT
petal_length,5.25..6.7  Set to be NULL
sepal_length,5.25..7.9 also set to be NULL
petal_length,5.25..6.7 also set to be NULL
Intersection set :  {124}
NOT
petal_width,0.1..1.85  Set to be NULL
petal_width,0.1..1.85 also set to be NULL
petal_width,0.1..1.95 also set to be NULL
petal_width,0.1..2.05 also set to be NULL
petal_width,0.1..2.15 also set to be NULL
petal_width,0.1..2.25 also set to be NULL
petal_width,0.1..2.35 also set to be NULL
petal_width,0.1..2.45 also set to be NULL
Intersection set :  set()


In [68]:
print("Time to run the algorithm (for Upper Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Iris/Rules/Iris_35_attcon_upper.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Upper Approximation):  13.484 Sec


['(sepal_length,4.85..7.9) & (petal_width,0.45..2.5) & (sepal_length,4.3..4.85) & (sepal_width,3.35..3.05) & (petal_length,1.45..6.7) -> (class,Iris-setosa)']
['(petal_length,4.25..4.55) & (petal_width,1.45..2.5) & (sepal_length,5.95..5.95) & (sepal_width,2.95..4.1) & (petal_width,0.1..1.45) -> (class,Iris-versicolor)']
['(sepal_width,2.95..4.1) & (sepal_length,6.25..6.35) & (petal_length,5.25..5.15) & (petal_width,1.85..1.85) -> (class,Iris-viginica)']
